In [3]:
import os
import cv2
import imutils
import operator
import numpy as np
import matplotlib.pyplot as plt
C_Root = './drive/MyDrive/Projects/Sudoku'

In [4]:
class Puzzle():
    def __init__(self, file_name):
        """Import and read the image."""
        self.file_name = file_name
        self.puzzle = cv2.imread(f'{C_Root}/RawPix/{self.file_name}', 0)

    def dist(self, p1, p2):
        """Returns the scalar distance between two points."""
        return np.sqrt(((p2[0] - p1[0]) ** 2) + ((p2[1] - p1[1]) ** 2))

    def infer_grid(self, img):
        """Find the grid that has the puzzles."""
        squares = []
        side = img.shape[:1]
        side = side[0] / 9
        for i in range(9):
            for j in range(9):
                p1 = (i * side, j * side) 
                p2 = ((i + 1) * side, (j + 1) * side) 
                squares.append((p1, p2))
        return squares

    def crop_and_warp(self, img, crop_rect):
        """Crops and warps a rectangular section from an image into a square of similar size."""

        tl, tr, br, bl = crop_rect[0], crop_rect[1], crop_rect[2], crop_rect[3]
        src = np.array([tl, tr, br, bl], dtype='float32')
        side = max([self.dist(br, tr), self.dist(tl, bl), self.dist(br, bl), self.dist(tl, tr)])
        dst = np.array([[0, 0], [side - 1, 0], [side - 1, side - 1], [0, side - 1]], dtype='float32')
        m = cv2.getPerspectiveTransform(src, dst)
        return cv2.warpPerspective(img, m, (int(side), int(side)))

    def find_corners(self, img):
        """Finds the 4 extreme corners of the largest contour in the image.
            Bottom-Left  : Smallest x - y value
            Bottom-Right : Largest  x + y value
            Top-Left     : Smallest x + y value
            Top-Right    : Largest  x - y value"""

        contours, h = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = sorted(contours, key=cv2.contourArea, reverse=True)
        polygon = contours[0]

        bottom_right, _ = max(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
        top_left, _ = min(enumerate([pt[0][0] + pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
        bottom_left, _ = min(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))
        top_right, _ = max(enumerate([pt[0][0] - pt[0][1] for pt in polygon]), key=operator.itemgetter(1))

        return [polygon[top_left][0], polygon[top_right][0], polygon[bottom_right][0], polygon[bottom_left][0]]


    def preprocess(self, img, if_dilate = False):
        """Gaussian Blur to reduce the noise in the image / smoothen the image. Technically, high frequency components are removed.
                1. The ksize should be an odd number tuple. Directly proportional to blur intensity.
                2. Standard Deviation of the Gaussian kernel.
           Thresholding for image segmentation - adaptive (different threshold for different areas of the image.
                1. The maxValue is dependent on the color format of the image. 
                2. The adaptive method is used to select the threshold.
                3. The thresholdType is used to control the color format of the output.
                4. The blockSize is used to control the smudgeness of the output. Should be odd and greater than one.
                5. The C is the contrast. Directly proportional to brightness."""
        pic = img
        pic = cv2.GaussianBlur(src = pic, ksize = (21, 21), sigmaX = cv2.BORDER_DEFAULT)
        pic = cv2.adaptiveThreshold(src = pic, maxValue = 255, adaptiveMethod = cv2.ADAPTIVE_THRESH_MEAN_C | cv2.ADAPTIVE_THRESH_MEAN_C,
                                    thresholdType = cv2.THRESH_BINARY_INV, blockSize = 9, C = 2)
        if if_dilate:
            pic = cv2.dilate(src = pic, kernel = np.array([[0, 1, 0], [1, 1, 1], [0, 1, 0]], dtype= np.uint8))
        return pic   

    def write_image(self, img, squares):
        """Write the transformed image after resizing the image to 252 x 252 px."""
        img = self.preprocess(img, if_dilate = False)
        img = cv2.resize(img, (252, 252))
        cv2.imwrite(f"{C_Root}/PrcPix/{self.file_name}", img) 

    def prepare_puzzle(self):
        processed = self.preprocess(self.puzzle, if_dilate = True)
        corners = self.find_corners(processed)
        cropped = self.crop_and_warp(self.puzzle, corners)
        squares = self.infer_grid(cropped)
        self.write_image(cropped, squares)
        
for filename in os.listdir(f'{C_Root}/RawPix'):
    board = Puzzle(file_name = filename)
    board.prepare_puzzle()